In [1]:
# Build the vocabulary from M most frequent words
# Use Counter object from the collections library to find M most common words
from collections import Counter

# The target size of the vocabulary
M = 3

# Counter could be used to build this dictionary from the source corpus
word_counts = {'happy': 5, 'because': 3, 'i': 2, 'am': 2, 'learning': 3, '.': 1}

# most_common gets the word >= M(3 )
vocabulary = Counter(word_counts).most_common(M)

# Remove the frequencies and leave just the words
vocabulary = [w[0] for w in vocabulary]

print(f"The new vocabulary containing {M} most frequent words: {vocabulary}\n")

The new vocabulary containing 3 most frequent words: ['happy', 'because', 'learning']



In [2]:
sentence = ['am','i','learning']
output_sentence = []
print(f"Input sentence : {sentence}")

for w in sentence:
  # test if word w is in vocabulary
  if w in vocabulary:
    output_sentence.append(w)
  else:
    output_sentence.append("<UNK>")

print(f"Output sentence : {output_sentence}")

Input sentence : ['am', 'i', 'learning']
Output sentence : ['<UNK>', '<UNK>', 'learning']


In [3]:
# Iterate through all word counts and print words with given frequency f
f = 3

word_counts = {'happy': 5, 'because': 3, 'i': 2, 'am': 2, 'learning':3, '.': 1}

for word,freq in word_counts.items():
  if freq == f:
    print(word)

because
learning


In [7]:
# many <unk> low perplexity
training_set = ['i', 'am', 'happy', 'because','i', 'am', 'learning', '.']
training_set_unk = ['i', 'am', '<UNK>', '<UNK>','i', 'am', '<UNK>', '<UNK>']

test_set = ['i', 'am', 'learning']
test_set_unk = ['i', 'am', '<UNK>']

M = len(test_set)
probability = 1
probability_unk = 1

# pre-calculated probabilities
bigram_probabilities = {('i', 'am'): 1.0, ('am', 'happy'): 0.5, ('happy', 'because'): 1.0, ('because', 'i'): 1.0, ('am', 'learning'): 0.5, ('learning', '.'): 1.0}
bigram_probabilities_unk = {('i', 'am'): 1.0, ('am', '<UNK>'): 1.0, ('<UNK>', '<UNK>'): 0.5, ('<UNK>', 'i'): 0.25}

# Got through the test set and calculate its bigram probability
for i in range(len(test_set) - 2 + 1):
  bigram = tuple(test_set[i : i + 2])
  probability = probability * bigram_probabilities[bigram]

  bigram_unk = tuple(test_set_unk[i : i + 2])
  probability_unk = probability_unk * bigram_probabilities_unk[bigram_unk]

# Calculate perplexity for both original test set and test set with <UNK>
perplexity = probability ** (-1/M)
perplexity_unk = probability_unk ** (-1/M)

print(f"Perplexity for the training set : ", {perplexity})
print(f"Perplexity for the training set with <UNK>: ", {perplexity_unk})

Perplexity for the training set :  {1.2599210498948732}
Perplexity for the training set with <UNK>:  {1.0}


### Smoothing

In [8]:
def add_K_smoothing_probability(k,vocabulary_size,n_gram_count,n_gram_prefix_count):
  numerator = n_gram_count + k
  denominator = n_gram_prefix_count + k * vocabulary_size
  return numerator / denominator

trigram_probabilities =  {('i', 'am', 'happy') : 2}
bigram_probabilities = {( 'i', 'am') : 10}
vocabulary_size = 5
k = 1

probability_known_trigram = add_K_smoothing_probability(k, vocabulary_size, trigram_probabilities[('i', 'am', 'happy')],
                                                        bigram_probabilities[( 'i', 'am')])
probability_unknown_trigram = add_K_smoothing_probability(k, vocabulary_size, 0, 0)

print(f"probability_known_trigram   : {probability_known_trigram}")
print(f"probability_unknown_trigram : {probability_unknown_trigram}")

probability_known_trigram   : 0.2
probability_unknown_trigram : 0.2


### Back-off

In [9]:
# Back-off is a model generalization method that leverages information from lower order n-grams
# in case information about the high order n-grams is missing. For example, if the probability
# of an trigram is missing, use bigram information and so on.
# pre-calculated probabilities of all types of n-grams
trigram_probabilities = {('i', 'am', 'happy'): 0}
bigram_probabilities = {( 'am', 'happy'): 0.3}
unigram_probabilities = {'happy': 0.4}

# This is the input trigram we need to estimate
trigram = ('are', 'you', 'happy')

# Find the last bigram and unigram of the input
bigram = trigram[1:3]
unigram = trigram[2]
print(f"besides the trigram {trigram} we also use bigram {bigram} and unigram ({unigram})\n")

# 0.4 is used as an example, experimentally found for web-scale corpuses when using the "stupid" back-off
lambda_factor = 0.4
probability_hat_trigram = 0

if trigram not in trigram_probabilities or trigram_probabilities[trigram] == 0:
  print(f"Probability for trigram {trigram} not found")

  if bigram not in bigram_probabilities or bigram_probabilities[bigram] == 0:
    print(f"Probability for bigram {bigram} not found")

    if unigram in unigram_probabilities:
      print(f"Probability for unigram {unigram} found\n")
      probability_hat_trigram = lambda_factor * lambda_factor * unigram_probabilities[unigram]

    else:
      probability_hat_trigram = 0

  else:
    probability_hat_trigram = trigram_probabilities[trigram]

  print(f"Probability for trigram {trigram} estimated as {probability_hat_trigram}")


besides the trigram ('are', 'you', 'happy') we also use bigram ('you', 'happy') and unigram (happy)

Probability for trigram ('are', 'you', 'happy') not found
Probability for bigram ('you', 'happy') not found
Probability for unigram happy found

Probability for trigram ('are', 'you', 'happy') estimated as 0.06400000000000002


### Interpolation

In [10]:
# pre-calculated probabilities of all types of n-grams
trigram_probabilities = {('i', 'am', 'happy'): 0.15}
bigram_probabilities = {( 'am', 'happy'): 0.3}
unigram_probabilities = {'happy': 0.4}

# The weights come from optimazation on a validation set
# The summation of all lambdas must be equal to one
lambda_1 = 0.8
lambda_2 = 0.15
lambda_3 = 0.05

# this is the input trigram we need to estimate
trigram = ('i', 'am', 'happy')

# Find the last bigram and unigram of the input
bigram = trigram[1:3]
unigram = trigram[2]
print(f"besides the trigram {trigram} we also use bigram {bigram} and unigram ({unigram})\n")

# in the production code, you would need to check if the probability n-gram dictionary contains the n-gram
probability_hat_trigram = lambda_1 * trigram_probabilities[trigram]
+ lambda_2 * bigram_probabilities[bigram]
+ lambda_3 * unigram_probabilities[unigram]

print(f"estimated probability of the input trigram {trigram} is {probability_hat_trigram}")

besides the trigram ('i', 'am', 'happy') we also use bigram ('am', 'happy') and unigram (happy)

estimated probability of the input trigram ('i', 'am', 'happy') is 0.12
